In [1]:
import sys
import time
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets


import tf_lab.autopredictors.scripts.virt_scan_data as vscan


from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension, \
                                                 shape_net_core_synth_id_to_category

from tf_lab.autopredictors.scripts import minhyuk_data

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions, \
                                           save_pc_prediction_stats
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [25]:
from tf_lab.autopredictors.evaluate import l1_loss_comparison_like_Angela
from tf_lab.autopredictors.scripts.fsdf_bin_parser import fsdf_bin_parser
from tf_lab.autopredictors.scripts.virt_scan_data import load_signed_distance_field
                    
category = 'Table'    
top_our_out_dir = '/orions4-zfs/projects/lins2/Lin_Virtual/Data/Output/PC2DF_All/' + category

syn_id = shape_net_category_to_synth_id()[category.lower()]

sdf_top_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/From_Matthias/shapenet_dim32_sdf/' + syn_id
angela_output_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/From_Matthias/predictions-test-small/epn/test/' + syn_id
angela_end = '.fsdf.bin'
angela_pred_files = [f for f in files_in_subdirs(angela_output_dir, angela_end + '$')]


n_examples = len(angela_pred_files)

total_l1 = np.zeros((n_examples, 2))
total_l1_with_unknown_mask = np.zeros((n_examples, 2))
total_l1_with_unknown_mask_range_3 = np.zeros((n_examples, 2))

threshold_range = 3.0
for i, ang_pred_file in enumerate(angela_pred_files):
    model_name = ang_pred_file.split('/')[-1][:-len(angela_end)]
    tokens = model_name.split('_')
    model_name = tokens[0]
    scan_id = tokens[2]
    angela_prediction = fsdf_bin_parser(ang_pred_file)
    
    our_pred_file = osp.join(top_our_out_dir, model_name + '___' + scan_id + '___pred_df.txt')
    gt_file = osp.join(top_our_out_dir, model_name + '___' + scan_id + '___gt_df.txt')
    _, unknown_space_mask = load_signed_distance_field(osp.join(sdf_top_dir, model_name + '__' + scan_id + '__.sdf') )
    
    our_pred_udf = np.loadtxt(our_pred_file).reshape(32, 32, 32)
    gt_udf = np.loadtxt(gt_file).reshape(32, 32, 32)
    
    total_l1[i, 0] = np.mean(np.abs(gt_udf - our_pred_udf))
    total_l1[i, 1] = np.mean(np.abs(gt_udf - angela_prediction))    
    
    total_l1_with_unknown_mask[i, 0] = l1_loss_comparison_like_Angela(gt_udf, our_pred_udf, unknown_space_mask)
    total_l1_with_unknown_mask[i, 1] = l1_loss_comparison_like_Angela(gt_udf, angela_prediction, unknown_space_mask)
    
    total_l1_with_unknown_mask_range_3[i, 0] = l1_loss_comparison_like_Angela_test(gt_udf, our_pred_udf, unknown_space_mask, threshold_range)
    total_l1_with_unknown_mask_range_3[i, 1] = l1_loss_comparison_like_Angela_test(gt_udf, angela_prediction, unknown_space_mask, threshold_range)    
    

In [26]:
evaluator = np.mean
print evaluator(total_l1[:,0]), evaluator(total_l1[:,1])
print evaluator(total_l1_with_unknown_mask[:,0]), evaluator(total_l1_with_unknown_mask[:,1])
print evaluator(total_l1_with_unknown_mask_range_3[:,0]), evaluator(total_l1_with_unknown_mask_range_3[:,1])


evaluator = np.median
print evaluator(total_l1[:,0]), evaluator(total_l1[:,1])
print evaluator(total_l1_with_unknown_mask[:,0]), evaluator(total_l1_with_unknown_mask[:,1])
print evaluator(total_l1_with_unknown_mask_range_3[:,0]), evaluator(total_l1_with_unknown_mask_range_3[:,1])

0.0990392061996 0.109946378675
0.0873738531253 0.0987364183873
0.33397697047 0.355882896922
0.0857514326222 0.098560853374
0.0721188311981 0.0837621379552
0.312101868867 0.337895253019
